In [46]:
import numpy as np
import scipy
from scipy.fftpack import fft2, ifft2
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import kron

In [47]:
# Initialize x-y mesh
x = np.linspace(-10, 10, 64, endpoint=False)
y = np.linspace(-10, 10, 64, endpoint=False)
X, Y = np.meshgrid(x, y)

# Define initial conditions
m = 1
alpha = 0
n = 64
u = (np.tanh(np.sqrt(X**2 + Y**2)) - alpha)*np.cos(m*np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
v = (np.tanh(np.sqrt(X**2 + Y**2)) - alpha)*np.sin(m*np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))

# Transform into Fourier domain
u0 = fft2(u).flatten()
v0 = fft2(v).flatten()

vec0_new = np.concatenate([u0, v0])
#vec0_new = np.hstack([(u0.reshape(n*n), v0.reshape(n*n))])

# Append initial conditions
#u0 = u0.reshape(-1, 1, order='F')
#v0 = v0.reshape(-1, 1, order='F')
#vec0 = np.concatenate((u0, v0))

def rhs1(t, vec, beta, n, KX, KY):
    """Right-hand side function to return Fourier transform of the solution"""
    u_hat = vec[:4096].reshape(n, n, order='F')
    v_hat = vec[4096:].reshape(n, n, order='F')

    # Transform out of Fourier domain
    u = ifft2(u_hat)
    v = ifft2(v_hat)

    u_nl = u - u**3 - u*v**2 + beta*(v*u**2 + v**3)
    v_nl = -beta*(u**3 + u*v**2) + v - v*u**2 - v**3

    u_t = fft2(u_nl) - 0.1*((KX**2)*u_hat + (KY**2)*u_hat)
    v_t = fft2(v_nl) - 0.1*((KX**2)*v_hat + (KY**2)*v_hat)

    u_t = u_t.reshape(n**2, order='F')
    v_t = v_t.reshape(n**2, order='F')
    rhs = np.concatenate((u_t, v_t), axis=0)

    return rhs

In [48]:
vec0_new.shape

(8192,)

In [49]:
t_span = np.linspace(0, 4, 9)
r1 = np.arange(0, n/2, 1)
r2 = np.arange(-n/2, 0, 1)
kx = (2*np.pi/20)*np.concatenate((r1, r2))
ky = kx.copy()
KX, KY = np.meshgrid(kx, ky)
beta = 1

# Timestep using the explicit Runge-Kutta method of order 4(5)
#sol = solve_ivp(diffusion_rhs,t_span=[0, 4],y0=y0,t_eval=tspan,method='RK45',args=params)
sol1 = solve_ivp(rhs1, [0, 4], y0=vec0_new, t_eval = t_span, method='RK45', args=(beta, n, KX, KY))
A1 = sol1.y

In [50]:
print(A1)

[[ 24.94003847+0.00000000e+00j  12.73268299-3.81634074e-15j
   -1.38095598-4.60674905e-15j ... -64.02389647-1.92707133e-14j
  -67.76356741-1.20381749e-14j -61.18058974-9.88753802e-15j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389

In [28]:
#[X,Y]=np.meshgrid(x,y)
##tspan[8] =25
#z = np.real(ifft2(sol1.y[0:4096,8].reshape(64,64)))
#print(np.shape(z))

# Generate the plot
#fig, ax = plt.subplots()
#cmap = ax.pcolor(X, Y, z.T)
#fig.colorbar(cmap)
#plt.show(fig)

# Problem 2

In [29]:
# Define Cheb
def cheb(N):
    if N == 0:
        D = 0.
        x = 1.
    else:
        n = np.arange(0, N+1)
        x = np.cos(np.pi*n/N).reshape(N+1, 1)
        c = (np.hstack(([2.], np.ones(N-1), [2.])) * (-1)**n).reshape(N+1, 1)
        X = np.tile(x, (1, N+1))
        dX = X - X.T
        D = np.dot(c, 1./c.T) / (dX + np.eye(N+1))
        D -= np.diag(np.sum(D.T, axis=0))  # Changed sum() to np.sum()
    
    return D, x.reshape(N+1)

In [30]:
# Define initial conditions
m = 1
alpha = 0
n = 30
N2 = (n+1)*(n+1)
# Create the Chebyshev differentiation matrix
D, x = cheb(n)

D[n,:]=0
D[0,:]=0

D2 = (np.dot(D, D))/((10)*(10))
y = x

# Scale Laplacian
I = np.eye(len(D2))
Lap = kron(D2, I) + kron(I, D2)


# Create the Chebyshev points
X, Y = np.meshgrid(x,y)
X = X*(20/2)
Y = Y*(20/2)

In [31]:
u = (np.tanh(np.sqrt(X**2 + Y**2)) - alpha)*np.cos(m*np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
v = (np.tanh(np.sqrt(X**2 + Y**2)) - alpha)*np.sin(m*np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))

# Append initial conditions
u1 = u.flatten()
v1 = v.flatten()
vec1 = np.concatenate([u1, v1])

def rhs2(t, vec, beta,N, Lap):
    """"Right-hand side function to solve our PDE"""
    u = vec[:N]
    v = vec[N:2*N]

    u_nl = u - u**3 - u*v**2 + beta*(v*u**2 + v**3)
    v_nl = -beta*(u**3 + u*v**2) + v - v*u**2 - v**3

    u_t = u_nl + 0.1*(Lap@u)
    v_t = v_nl + 0.1*(Lap@v)

    rhs = np.concatenate((u_t, v_t), axis=0)

    return rhs



In [32]:
# Timestep using the explicit Runge-Kutta method of order 4(5)
sol2 = scipy.integrate.solve_ivp(lambda t, vec: rhs2(t, vec, beta,N2, Lap), [0, 4], np.squeeze(vec1), t_eval = t_span)
A2 = sol2.y

In [33]:
A2

array([[ 0.70358468,  0.27678435, -0.21775865, ..., -0.79689015,
        -0.40972859,  0.07776933],
       [ 0.73241275,  0.47188952,  0.07344742, ..., -0.96577657,
        -0.78500366, -0.4261521 ],
       [ 0.81058026,  0.37605887, -0.11123233, ..., -0.84008598,
        -0.49565779, -0.03085913],
       ...,
       [ 0.58562756,  0.91352592,  0.97914313, ..., -0.50294695,
        -0.84298442, -0.97634716],
       [ 0.6808609 ,  0.87018536,  0.97997159, ..., -0.16453512,
        -0.5878894 , -0.88455009],
       [ 0.71061143,  0.96093661,  0.97601586, ..., -0.60413504,
        -0.91222169, -0.99697897]])